In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import re

In [2]:
stopwords  = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection', sep= '\t')
data.columns = ['label','body_text']

In [3]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")),3)*100

data['body_len'] = data['body_text'].apply(lambda x:len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x:count_punct(x))

In [4]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+',text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [5]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())],axis = 1)

In [6]:
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,24,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,39,15.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 50, max_depth = 20, n_jobs = 2)
rf_model = rf.fit(X_train, y_train)

In [11]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[:10]

[(0.09107059887263706, 'body_len'),
 (0.0450818500669498, 3135),
 (0.03160656186896467, 1804),
 (0.027650504782530658, 6749),
 (0.026438607077386857, 5727),
 (0.02135187476531353, 6288),
 (0.020914024370048336, 4799),
 (0.018206398500620124, 1361),
 (0.016075228706893242, 5081),
 (0.015894882480150828, 2032)]

In [12]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam',average='binary')

In [17]:
print('Precison: {} / Recall: {} / Accuracy: {}'.format(round(precision,3),
                                                        round(recall,3),
                                                        round((y_pred == y_test).sum()/len(y_pred),3)))
      

Precison: 1.0 / Recall: 0.56 / Accuracy: 0.941


In [18]:
#Deduction:
#1. Precision:1 = when a model identified something as spam, it actually was spam
#2. Recall:0.56 = of all the actual spam, the model was able to predict only so number of times
#3. Accuracy:.94 = the model was able to correctly label 94% of sms correctly